# Simple Stack Comparison
- Take a pair of Z-stacks from animals expressing functional indicators, with sustained sensory stimuli (i.e., "steady state" functional stacks)
- Pass the paths to the two stacks (assume no alignment is needed)
- Normalize each, to low and high percentile targets
- Take the difference, such that the whole thing scales between -1 to 1
- Visualize on napari (to guide experiments immediately following)
- Save the difference stack as h5, or possible as tiff or movie

In [75]:
import numpy as np
from matplotlib import pyplot as plt
import napari
import colorcet as cc
from scipy.signal import convolve
import flammkuchen as fl

In [89]:
## Convert colormap to use as napari-digestable dict

cet_cmap = cc.cm.CET_C1
colors = np.vstack([cet_cmap(i) for i in range(256)])

new_colormap = {
    'colors': colors,
    'name': cet_cmap.name,
    'interpolation': 'linear'
}

In [63]:
## Generate sham data (for now)

xx, yy, zz = np.meshgrid(np.linspace(-1,1,400), np.linspace(-1,1,200), np.linspace(-1,1,400)) 

stack0 = (np.sin(xx*10 + zz*np.sin(yy*10))) * (np.sqrt(xx**2 + yy**2 + zz**2) < 0.8) + np.random.rand(*xx.shape)*5
stack1 = (np.cos(xx*23 + yy*np.cos(zz*13))) * (np.sqrt(xx**2 + yy**2 + zz**2) < 0.8) + np.random.rand(*xx.shape)*5

del xx, yy, zz

In [64]:
# normalize, take difference
nstack0 = (stack0 - np.percentile(stack0, 1)) / (np.percentile(stack0, 99) - np.percentile(stack0, 1))
nstack1 = (stack1 - np.percentile(stack1, 1)) / (np.percentile(stack1, 99) - np.percentile(stack1, 1))
dstack = nstack0 - nstack1

In [65]:
del stack0, stack1, nstack0, nstack1

In [90]:
# box filtering
box_size = 5
s_dstack = np.asarray([convolve(frame, np.ones((box_size, box_size))/box_size**2, 'same') for frame in dstack])

In [91]:
max_cont = 0.5
viewer0 = napari.view_image(
                    s_dstack, 
                    colormap=new_colormap,
                    contrast_limits=(-max_cont, max_cont)
          )

In [92]:
# save h5
save_path = 'test_diff_stack'
fl.save(save_path+'.h5', {'stack': s_dstack})

In [79]:
# load test
s_dstack = fl.load(save_path+'.h5', '/stack')

In [84]:
temp = cc.cm.CET_D1

In [86]:
temp.name

'diverging_bwr_40_95_c42'